In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as scs
import matplotlib.pyplot as plt
import sqlite3
import json
import joblib
import pickle
import warnings
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import collections
import numpy as np
import os
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from utils.data import MatchDataGenerator, df_to_dict
from utils.basic_layers import MLP, EmbeddingLayer
from utils.features import SparseFeature, SequenceFeature
from utils.match import Annoy, generate_seq_feature_match, gen_model_input
from utils.metrics import topk_metrics
from utils.trainer import MatchTrainer

warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
torch.manual_seed(42);

In [2]:
data = pd.read_csv('train_10k.csv')



In [3]:
data = data.drop(columns=['release', 'artist_name', 'artist_country', 'artist_city', 'year', 'title', 'genre'])

In [4]:
data.head()

,user_id,song_id,play_count
0,6fbf6970611d01e10aebeab374f461116155867e,SOPVPCY12A81C23555,1
1,fa8a8753518e6c2d3713990dc2a172ea17000b80,SOBSMEQ12AB018282F,1
2,c9fdf63587a7a963e383ea2f1b58d1014377caab,SONQEYS12AF72AABC9,1
3,e329cc2012d31242297d294fa0279b79a1bd5cc7,SOHTAXD12A8C141E75,1
4,2a9178398fa6377a340d5b9b6be87de32b4059a2,SOAWWJW12AB01814F5,2


In [6]:
# data["cat_id"] = data["genres"].apply(lambda x: x.split("|")[0])
user_col, item_col = "user_id", "song_id"
sparse_features = ['user_id', 'song_id']
# sparse_features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip', "cat_id"]

In [7]:
save_dir = './saved/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
    
# print(f'Before encoding: \n {data[sparse_features].tail()}')

feature_max_idx = {}
for feature in sparse_features:
    encoder = LabelEncoder()
    data[feature] = encoder.fit_transform(data[feature]) + 1 # лучше энкодить не с 0, особенно в sequential NN
    feature_max_idx[feature] = data[feature].max() + 1
    if feature == user_col:
        user_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(encoder.classes_)}
    if feature == item_col:
        item_map = {encode_id + 1: raw_id for encode_id, raw_id in enumerate(encoder.classes_)}
np.save(save_dir + "raw_id_maps.npy", (user_map, item_map))

# print(f'After encoding: \n {data[sparse_features].tail()}')

In [8]:
# user_cols = ["user_id", "gender", "age", "occupation", "zip"]
user_cols = ["user_id"]

item_cols = ['song_id']
user_profile = data[user_cols].drop_duplicates('user_id')
item_profile = data[item_cols].drop_duplicates('movie_id')

In [11]:
df_train, df_test = generate_seq_feature_match(data,
                                               user_col,
                                               item_col,
                                               item_attribute_cols=[],
                                               sample_method=1,
                                               mode=0,
                                               neg_ratio=3,
                                               min_item=3)

generate sequence features: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 260.15it/s]


n_train: 39440, n_test: 70
0 cold start users droped 


In [12]:
x_train = gen_model_input(df_train, user_profile, user_col, item_profile, item_col, seq_max_len=50)
x_test = gen_model_input(df_test, user_profile, user_col, item_profile, item_col, seq_max_len=50)

y_train = x_train["label"]
y_test = x_test["label"]

In [13]:
user_features = [
    SparseFeature(feature_name, vocab_size=feature_max_idx[feature_name], embed_dim=16) for feature_name in user_cols
]

user_features += [
    SequenceFeature("hist_song_id",
                    vocab_size=feature_max_idx["song_id"],
                    embed_dim=16,
                    pooling="mean",
                    shared_with="song_id")
]

item_features = [
    
    SparseFeature(feature_name, vocab_size=feature_max_idx[feature_name], embed_dim=16) for feature_name in item_cols
]

In [14]:
all_item = df_to_dict(item_profile)

test_user = x_test
data_generator = MatchDataGenerator(x=x_train, y=y_train)
train_dl, test_dl, item_dl = data_generator.generate_dataloader(test_user, all_item, batch_size=128)

In [15]:
class YoutubeDNN(torch.nn.Module):
    """
    The match model mentioned in `Deep Neural Networks for YouTube Recommendations` paper.
    It's a DSSM match model trained by global softmax loss on list-wise samples. 
    In origin paper, item dnn tower is missing.
    Args:
        user_features (list[Feature Class]): training by the user tower module.
        item_features (list[Feature Class]): training by the embedding table, it's the item id feature.
        neg_item_feature (list[Feature Class]): training by the embedding table, it's the negative items id feature.
        user_params (dict): the params of the User Tower module, 
        keys include:`{"dims":list, "activation":str, "dropout":float, "output_layer":bool`}.
        temperature (float): temperature factor for similarity score, default to 1.0.
    """

    def __init__(self, user_features, item_features, neg_item_feature, user_params, temperature=1.0):
        super().__init__()
        self.user_features = user_features
        self.item_features = item_features
        self.neg_item_feature = neg_item_feature
        self.temperature = temperature
        self.user_dims = sum([fea.embed_dim for fea in user_features])
        self.embedding = EmbeddingLayer(user_features + item_features)
        self.user_mlp = MLP(self.user_dims, output_layer=False, **user_params)
        self.mode = None

    def forward(self, x):
        user_embedding = self.user_tower(x)
        item_embedding = self.item_tower(x)
        if self.mode == "user":
            return user_embedding
        if self.mode == "item":
            return item_embedding

        y = torch.mul(user_embedding, item_embedding).sum(dim=2)
        y = y / self.temperature
        return y

    def user_tower(self, x):
        if self.mode == "item":
            return None
        # [batch_size, num_features * deep_dims]
        input_user = self.embedding(x, self.user_features, squeeze_dim=True)
        # [batch_size, 1, embed_dim]
        user_embedding = self.user_mlp(input_user).unsqueeze(1)
        user_embedding = F.normalize(user_embedding, p=2, dim=2)
        if self.mode == "user":
            return user_embedding.squeeze(1)
        return user_embedding

    def item_tower(self, x):
        if self.mode == "user":
            
            return None
        #[batch_size, 1, embed_dim]
        pos_embedding = self.embedding(x, self.item_features, squeeze_dim=False)
        pos_embedding = F.normalize(pos_embedding, p=2, dim=2)
        # inference embedding mode
        if self.mode == "item":
            # [batch_size, embed_dim]
            return pos_embedding.squeeze(1)
        #[batch_size, n_neg_items, embed_dim]
        neg_embeddings = self.embedding(x, self.neg_item_feature,
                                        squeeze_dim=False).squeeze(1)
        neg_embeddings = F.normalize(neg_embeddings, p=2, dim=2)
        # [batch_size, 1 + n_neg_items, embed_dim]
        return torch.cat((pos_embedding, neg_embeddings), dim=1)

In [16]:
df_train, df_test = generate_seq_feature_match(data,
                                               user_col,
                                               item_col,
                                               item_attribute_cols=[],
                                               sample_method=1,
                                               mode=2,
                                               neg_ratio=3,
                                               min_item=0)
x_train = gen_model_input(df_train, user_profile, user_col, item_profile, item_col, seq_max_len=50)
y_train = np.array([0] * df_train.shape[0])
x_test = gen_model_input(df_test, user_profile, user_col, item_profile, item_col, seq_max_len=50)

# user_cols = ['user_id', 'gender', 'age', 'occupation', 'zip']
user_cols = ['user_id']

user_features = [SparseFeature(name, vocab_size=feature_max_idx[name], embed_dim=16) for name in user_cols]
user_features += [
    SequenceFeature("hist_song_id",
                    vocab_size=feature_max_idx["song_id"],
                    embed_dim=16,
                    pooling="mean",
                    shared_with="song_id")
]

item_features = [SparseFeature('song_id', vocab_size=feature_max_idx['song_id'], embed_dim=16)]
neg_item_feature = [
    SequenceFeature('neg_items',
                    vocab_size=feature_max_idx['song_id'],
                    embed_dim=16,
                    pooling="concat",
                    shared_with="song_id")
]

all_item = df_to_dict(item_profile)
test_user = x_test

dg = MatchDataGenerator(x=x_train, y=y_train)
train_dl, test_dl, item_dl = dg.generate_dataloader(test_user, all_item, batch_size=512)



generate sequence features: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 1489.95it/s]


n_train: 9860, n_test: 70
0 cold start users droped 


In [17]:
def match_evaluation(user_embedding, item_embedding, test_user, all_item, user_col='user_id', item_col='song_id',
                     raw_id_maps="./raw_id_maps.npy", topk=10):
    
    # Fit Annoy tree on item embeddings
    annoy = Annoy(n_trees=10)
    annoy.fit(item_embedding)

    # For each user get top-k similar items
    user_map, item_map = np.load(raw_id_maps, allow_pickle=True)
    match_res = collections.defaultdict(dict)
    for user_id, user_emb in zip(test_user[user_col], user_embedding):
        items_idx, items_scores = annoy.query(v=user_emb, n=topk)
        match_res[user_map[user_id]] = np.vectorize(item_map.get)(all_item[item_col][items_idx])

    # Get ground truth
    data = pd.DataFrame({user_col: test_user[user_col], item_col: test_user[item_col]})
    data[user_col] = data[user_col].map(user_map)
    data[item_col] = data[item_col].map(item_map)
    user_pos_item = data.groupby(user_col).agg(list).reset_index()
    ground_truth = dict(zip(user_pos_item[user_col], user_pos_item[item_col]))

    # Compute top-k metrics
    
    y_true = ground_truth
    y_pred = match_res

    l = list(range(1, topk+1))
    out = topk_metrics(y_true=ground_truth, y_pred=match_res, topKs=l)
    return y_pred, y_true, out

In [18]:
model = YoutubeDNN(user_features, item_features, neg_item_feature, 
                   user_params={"dims": [128, 64, 16]}, temperature=0.02)

trainer = MatchTrainer(model,
                       mode=2,
                       
                       optimizer_params={
                           "lr": 1e-2,
                           "weight_decay": 1e-5
                       },
                       n_epoch=24,
                       device='cpu',
                       model_path=save_dir)

trainer.fit(train_dl)

print("inference embedding")
user_embedding = trainer.inference_embedding(model=model, mode="user", data_loader=test_dl, model_path=save_dir)
item_embedding = trainer.inference_embedding(model=model, mode="item", data_loader=item_dl, model_path=save_dir)
# match_evaluation(user_embedding, item_embedding, test_user, all_item, topk=100, raw_id_maps="./saved/raw_id_maps.npy")


epoch: 0


train: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 30.73it/s, loss=13.6]


epoch: 1


train: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 36.54it/s, loss=10.6]


epoch: 2


...

train: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 36.85it/s, loss=1.49]


epoch: 23


train: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 37.05it/s, loss=1.24]


inference embedding


item inference: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 34.77it/s]


In [19]:
item_embedding

tensor([[-0.3573, -0.1890,  0.2433,  ...,  0.2958, -0.2846, -0.3350],
        [ 0.1236, -0.1122, -0.0582,  ...,  0.1327, -0.0906, -0.1197],
        [ 0.1914,  0.1711, -0.6693,  ...,  0.1815,  0.1360, -0.0446],
        ...,
        [-0.2599, -0.1943, -0.2429,  ..., -0.2332, -0.2364, -0.2364],
        [-0.2309,  0.2465,  0.3711,  ...,  0.2151,  0.2085, -0.2146],
        [-0.1706, -0.2636,  0.1105,  ...,  0.1664, -0.2664, -0.2228]])

In [20]:
y_pred, y_true, ou = match_evaluation(user_embedding, item_embedding, test_user, all_item, topk=30, raw_id_maps="./saved/raw_id_maps.npy")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [25]:
prec_youtube = ou['Precision']
rec_youtube = ou['Recall']
map_youtube = ou['MAP']
mar_youtube = ou['MAR']
ndcg_k_youtube = ou['ndcg_k']
coverage = ou['coverage']
personalization = ou['personalization']
novelty = ou['novelty']

In [ ]:
path_numpy_data = 'metrics/youtube_prec'
np.save(path_numpy_data, prec_youtube)

In [ ]:
path_numpy_data = 'metrics/youtube_rec'
np.save(path_numpy_data, rec_youtube)

In [ ]:
path_numpy_data = 'metrics/youtube_map'
np.save(path_numpy_data, map_youtube)

In [ ]:
path_numpy_data = 'metrics/youtube_mar'
np.save(path_numpy_data, mar_youtube)

In [ ]:
path_numpy_data = 'metrics/youtube_ndcg'
np.save(path_numpy_data, ndcg_k_youtube)